In [2]:
import pandas as pd
%matplotlib inline
%run Pipeline//upload_and_vizualize 
%run Pipeline//classify_and_evaluate 
%run Pipeline//aux
%run Pipeline//ULAB_ML_Pipeline
%run processing
import numpy as np 
import matplotlib.pyplot as plt
import pylab
import sys
import random
import sklearn as sk 
import json 
import re
from multiprocessing import Pool
from functools import partial
from time import time
from sklearn import svm, ensemble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import train_test_split, KFold
from sklearn.preprocessing import *
from sklearn.feature_selection import RFE
from sklearn.grid_search import ParameterGrid
from sklearn.svm import LinearSVC
from sklearn.metrics import *
import csv
from errno import EEXIST
from os import makedirs,path
from datetime import datetime as dr
from datetime import date


ERROR:root:File `'Pipeline//aux.py'` not found.


SyntaxError: invalid syntax (ULAB_ML_Pipeline.py, line 204)

In [3]:
import csv
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import pylab as pl
from upload_and_vizualize import camel_to_snake
from datetime import datetime as dt
from datetime import date 

def get_desired_features(dict, desired_features):
    '''
    This function creates a list of desired column numbers that are representative of columns
    with desired features
    Inputs:
        dict = dictionary
        desired_features = list of desired features
    Output:
        list with column numbers
    '''

    features = [key for key,value in dict.items() if value in desired_features]
    return features

def check_na(df):
    '''
    Given a pandas dataframe this function will create a crosstab table for all columns and indicate which
    have null values
    '''
    df_lng = pd.melt(df)
    null_vars = df_lng.value.isnull()
    return pd.crosstab(df_lng.variable, null_vars)


def check_diff(df1,df2):
    '''
    Given two pandas dataframes, this function will give a count of the values with value differences
    '''

    diff = set(df1.index)-set(df2.index)
    return(len(diff))

def describe_extremes(df,column, greater_than):
    '''
    Given a pandas dataframe and a column of choice, this function will return information about
    extreme valuse above a certain normal value.
    Inputs:
        df = pandas dataframe
        column = column of choice
        greater_than = value that signifies the lower bound of an extereme value
    Outputs:
        prints distribution of column values greater than given lower bound and its percentage of
        the whole set
    '''

    column_str = str(column)
    very_high = df[column_str] > greater_than
    print(df[column_str].debt_ratio.describe())
    print(len(df[column_str])/len(df))

def add_categoricals(dict, new_categories):
    '''
    Given a dictionary of column features.  This will add new column features to that dictionary
    '''

    count=1
    for cat in new_categories:
        dict[len(dict)+count] = cat

def update_keys(dict):
    '''
    If an index is taken from a dataframe.  This will update the dictionary of columns accordingly
    '''
    return {key-1:value for key,value in dict.items()}


def add_dummies(df,need_dummies):
    '''
    Given a dataframe and columns that need to have dummy variables created from them. This will add
    dummy columns based on those values.
    Inputs:
        df = dataframe
        need_dummies = column names that need dummies
    Outputs:
        dataframe with new dummy variables
    '''
    new_cols = pd.DataFrame()
    for col in need_dummies:
        #print(np.array(df[col]))
        new_cols = pd.concat([new_cols.reset_index(drop=True),pd.get_dummies(df[col])],axis=1)

    df_w_dummies = pd.concat([df,new_cols],axis=1).reset_index(drop=True)
    return df_w_dummies

def convert_to_datetime(series_row, date_format):
    if str(series_row) == 'nan':
        return float('nan')
    return dt.strptime(series_row, date_format)

def convert_to_weekday(series_row, output):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    if output == 'day_num':
        return date.weekday(series_row)
    output_dict = {"weekday":'%A', "month_name":'%B',"month_num":'%m',"year":"%Y"}
    return date.strftime(series_row,output_dict[output])

def convert_to_bool(df, column, conversion):
    return df[column].replace(conversion)

def add_date_cols(df, date_columns, date_format, date_types):
    '''
    date_types = list of datetime output indicators
    '''
    if 'month' in date_types:
        print(date_types)
        date_types = [v for v in date_types if v != 'month']
        date_types.extend(['month_name','month_num'])
    for date_column in date_columns: 
        df[date_column+'_datetime'] = df[date_column].apply(convert_to_datetime, date_format=date_format)
        for new_col in date_types:
            df[date_column+'_'+new_col] = df[date_column+'_datetime'].apply(convert_to_weekday, output=new_col)
            
def get_occupied_frame(df, date_columns, date_format, date_types, bool_param = None, occ_column = None, conversion= None):
    add_date_cols(df,date_columns, date_format, date_types)
    if bool_param != None:
        df[occ_column] = convert_to_bool(df,occ_column,conversion)
        return df[df[occ_column] == bool_param]

In [4]:
FACILITY = 'data/ICIS-AIR_FACILITIES.csv'
VIOLATION = 'data/ICIS-AIR_VIOLATION_HISTORY.csv'
INSPECTION = 'data/ICIS-AIR_FCES_PCES.csv'
FORMALACT = 'data/ICIS-AIR_FORMAL_ACTIONS.csv'
INFORMALACT = 'data/ICIS-AIR_INFORMAL_ACTIONS.csv'
STACKTEST = 'data/ICIS-AIR_STACK_TESTS.csv'
TITLEV = 'data/ICIS-AIR_TITLEV_CERTS.csv'

#facilities = read_file('ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv')
#pd_violations = read_file('ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
#pd_inspection = read_file('ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')
#pd_formalact = read_file('ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv')
#pd_informalact = read_file('ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv')
#pd_stacktest = read_file('ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv')
#pd_titlev = read_file('ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv')

In [5]:
## FROM PREPROCESSING ##

#interest_var = ['PGM_SYS_ID','ACTIVITY_ID','AGENCY_TYPE_DESC','STATE_CODE','AIR_LCON_CODE','COMP_DETERMINATION_UID','ENF_RESPONSE_POLICY_CODE','PROGRAM_CODES']
def convert_to_datetime(series_row, date_format):
    if str(series_row) == 'nan':
        return float('nan')
    return dt.strptime(series_row, date_format)

def convert_to_year(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.year)

def to_date_time(df, date_format, date_col):
    #add datetime column
    df[date_col] = df[date_col].apply(convert_to_datetime, date_format=date_format)
    df[date_col+'_year'] = df[date_col].apply(convert_to_year)

    return df

def filter_date(df, date_format, date_col, start=None, end=None):
    df = to_date_time(df, date_format, date_col)
    
    
    if start:
        timestart = dt.strptime(start,"%Y/%m/%d")
        #print(start)
        df = df[df[date_col] >= timestart ]
        #print(df.head())
    if end:
        timeend = dt.strptime(end,"%Y/%m/%d")
        #print(end)
        df = df[df[date_col] <= timeend ]
        #print(df.head())
    
    return df

def filter_col(df, fac_id, features, date_col):
    #filter needed
    df = df[[fac_id] + [date_col] + [date_col+'_year'] + features]
    return df

In [6]:
def replace_with_value(data_file, variables, values):
	'''
	'''
	for variable in variables:
		value = values[variables.index(variable)]
		data_file[variable] = data_file[variable].fillna(value)

In [7]:
## CONFIG DATA ##
START_DATE= '2007/01/01'
END_DATE = '2016/12/31'
fac_id = 'PGM_SYS_ID'
df_dict ={'violation': {'data_file': 'ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv',
                        'interest_var': ['AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_RESOLVED_DATE'],
                         'date_col': 'HPV_DAYZERO_DATE',
                       'date_format':'%m-%d-%Y'},
          
           'inspection': {'interest_var': ['STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES'],
                         'date_col': 'ACTUAL_END_DATE',
                         'date_format':'%m-%d-%Y'},
          
           'stacktest': {'interest_var':['COMP_MONITOR_TYPE_CODE','POLLUTANT_CODES','AIR_STACK_TEST_STATUS_CODE'],
                        'date_col': 'ACTUAL_END_DATE',
                        'date_format':'%m/%d/%Y'},
          
           'titlev':{'interest_var':['COMP_MONITOR_TYPE_CODE','FACILITY_RPT_DEVIATION_FLAG'],
                        'date_col': 'ACTUAL_END_DATE',
                    'date_format':'%m/%d/%Y'},
          
           'formalact':{'interest_var':['ENF_TYPE_CODE','PENALTY_AMOUNT'],
                        'date_col': 'SETTLEMENT_ENTERED_DATE',
                       'date_format':'%m/%d/%Y'},
          
           'informalact':{'interest_var':['ENF_TYPE_CODE'],
                        'date_col': 'ACHIEVED_DATE',
                         'date_format':'%m/%d/%Y'}}

#general_df = pd.DataFrame()

In [8]:
for i, v in df_dict.items():
    print(v['date_col'])

HPV_DAYZERO_DATE
ACTUAL_END_DATE
ACTUAL_END_DATE
ACTUAL_END_DATE
SETTLEMENT_ENTERED_DATE
ACHIEVED_DATE


In [9]:
def general_read_file(df_dict, start_date, end_date):
    #df = pd.DataFrame()
    
    violation = []
    inspection = []
    stacktest = []
    titlev = []
    formalact = []
    informalact = []
    
    for table, var in df_dict.items():
        date_col = var['date_col']
        features = var['interest_var']
        DATE_FORMAT = var['date_format']
        
        if table == 'violation':
            violation = read_file(VIOLATION)
            violation = violation[violation['ENF_RESPONSE_POLICY_CODE'] != 'FRV']
            
            violation = filter_date(violation, DATE_FORMAT, date_col, start=start_date, end=end_date)
            violation = filter_col(violation, fac_id, features, date_col)
        
        elif table == 'inspection':
            inspection = read_file(INSPECTION)
            
            inspection = filter_date(inspection, DATE_FORMAT, date_col, start=start_date, end=end_date)
            inspection = filter_col(inspection, fac_id, features, date_col)
        
        elif table == 'titlev':
            titlev = read_file(TITLEV)
            
            titlev = filter_date(titlev, DATE_FORMAT, date_col, start=start_date, end=end_date)
            titlev = filter_col(titlev, fac_id, features, date_col)
        
        elif table == 'stacktest':
            stacktest = read_file(STACKTEST)
            
            stacktest = filter_date(stacktest, DATE_FORMAT, date_col, start=start_date, end=end_date)
            stacktest = filter_col(stacktest, fac_id, features, date_col)
        
        elif table == 'formalact':
            formalact = read_file(FORMALACT)
            
            formalact = filter_date(formalact, DATE_FORMAT, date_col, start=start_date, end=end_date)
            formalact = filter_col(formalact, fac_id, features, date_col)
        
        elif table == 'informalact':
            informalact = read_file(INFORMALACT)
            
            informalact = filter_date(informalact, DATE_FORMAT, date_col, start=start_date, end=end_date)
            informalact = filter_col(informalact, fac_id, features, date_col)
        
    return (violation, inspection, titlev, stacktest, formalact, informalact)

In [10]:
## READ INITIAL FILTERED FILE ##
violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)

In [11]:
## FROM PREPROCESSING, MODIFIED ##
def add_dummy(df, variable_list, sep_char = None, drop_one=False, drop_original=False):
    '''
    Input: 
        - df: pandas dataframe
        - variable_list: a list of variables to dummitize
        - drop_one: whether to drop first dummy
        - drop_original: whether to drop original categorical variable
    Output: dataframe with tht dummy variables added
    '''
    for variable in variable_list:
        if sep_char:
            df_dummy = df[variable].str.get_dummies(sep=sep_char)
            df_dummy.columns = [variable+ '_' +str(col) for col in df_dummy.columns]

        else:
            df_dummy = pd.get_dummies(df[variable], drop_first=drop_one, prefix = variable)
        
        df = pd.concat([df, df_dummy], axis=1)
        if drop_original:
            df = df.drop(variable, 1)
    return (df, df_dummy.columns)

In [12]:
violation.isnull().sum()

PGM_SYS_ID                     0
HPV_DAYZERO_DATE               0
HPV_DAYZERO_DATE_year          0
AGENCY_TYPE_DESC               0
AIR_LCON_CODE               7620
ENF_RESPONSE_POLICY_CODE       0
POLLUTANT_CODES              290
PROGRAM_CODES                  0
HPV_RESOLVED_DATE           2229
dtype: int64

In [13]:
viol = violation
viol

,PGM_SYS_ID,HPV_DAYZERO_DATE,HPV_DAYZERO_DATE_year,AGENCY_TYPE_DESC,AIR_LCON_CODE,ENF_RESPONSE_POLICY_CODE,POLLUTANT_CODES,PROGRAM_CODES,HPV_RESOLVED_DATE
5,CT0000000900508907,2010-03-17,2010,State,NaN,HPV,300000329,CAAFESOP,11-02-2010
6,CT0000000900100078,2007-08-08,2007,State,NaN,HPV,300000329,CAAFESOP,09-24-2009
7,CT0000000901501125,2013-04-26,2013,State,NaN,HPV,10193,CAATVP,06-25-2014
9,CT0000000900109058,2008-01-17,2008,State,NaN,HPV,300000005,CAAFESOP,01-06-2009
13,CT0000000901100054,2008-04-08,2008,State,NaN,HPV,10461 300000005,CAATVP,12-10-2008
16,CT0000000901100038,2009-10-01,2009,State,NaN,HPV,300000005,CAASIP,03-25-2011
18,CT0000000900900016,2010-04-26,2010,State,NaN,HPV,300000329,CAANSPS CAANSR CAASIP CAATVP,10-15-2012
24,MA0000002512000866,2010-03-01,2010,State,NaN,HPV,10461,CAASIP CAATVP,12-31-2011
29,CT0000000900909910,2010-07-27,2010,State,NaN,HPV,300000298,CAANSR,08-05-2011
30,CT0000000900909936,2010-05-18,2010,State,NaN,HPV,300000005,CAASIP CAATVP,02-01-2011


In [14]:
def aggr_dummy_cols(df, final_df, colnames, mode = None):
    for col in colnames:
        
        df['id_+_date'] = df.PGM_SYS_ID +'_'+ df.HPV_DAYZERO_DATE_year
        cross = pd.crosstab(df['id_+_date'], columns=df[col])
        
        if mode == 'cat':
            cross.columns = [colm + '_' + str(col) for colm in cross.columns]

        elif mode == 'dum':
            cross = cross.drop(0, axis = 1)
            cross.columns = [col for colm in cross.columns]
            
        #SIMPLIFY THIS!!
        elif mode == 'bim':
            cross = cross.drop('N', axis = 1)
            cross.columns = [col for colm in cross.columns]
        
        #ADD PROCESSING CONT_VAR
        
        else:
            cross.columns = [cross.columns.name for col in cross.columns]

        cross.columns.name = None
        cross.reset_index(inplace=True)
        re_separate = r'(.[^_]*)_(.*)'
        sep = lambda x: pd.Series([i for i in re.split(re_separate,x)])
        cross_id_year = cross['id_+_date'].apply(sep)
        cross =pd.concat([cross_id_year.rename(columns={1:'PGM_SYS_ID',2:'HPV_DAYZERO_DATE_year'}), cross], axis=1)
        cross.drop([0, 3, 'id_+_date'], axis = 1, inplace = True)

        
        if final_df.empty:
            final_df = final_df.append(cross)
        else:
            final_df = pd.merge(final_df, cross, how = 'left', on = 'PGM_SYS_ID')
            
    return final_df

In [15]:
## FROM ULAB PIPELINE ##

def generate_continous_variable(data_file, variable_list):
	'''
	function that can take a categorical variable and create 
	binary variables from it
	'''
	for variable in variable_list:
		list_values = list(data_file.groupby(variable).groups.keys())
		for i,value in enumerate(list_values):
			data_file[variable].replace(value,i)

	return data_file 

In [16]:
def process_violation(violation_df, start_year, end_year):
    final_df = pd.DataFrame()
    
    outcome = ['ENF_RESPONSE_POLICY_CODE']
    cat_var = ['AGENCY_TYPE_DESC', 'AIR_LCON_CODE']
    dum_var = ['PROGRAM_CODES', 'POLLUTANT_CODES']
    
    df = violation_df[violation_df['HPV_DAYZERO_DATE_year'] >= start_year]
    df = df[df['HPV_DAYZERO_DATE_year'] <= end_year]
    
    ## Replace NaN with 'None' (string) --> Making 'None' it's own category
    nan = df.columns[df.isnull().any()].tolist()
    values = ['None']*len(nan)
    replace_with_value(df, nan, values)
    
    ## Dummitize 
    df, colnames_out = add_dummy(df, outcome, drop_original = True)
    df, colnames_dum = add_dummy(df, dum_var, sep_char = ' ')
    
    #colnames_dum needs to be given previx to diff between viol_program codes vs inspection
    print(colnames_out)
    print(colnames_dum)
    
    final_df = aggr_dummy_cols(df, final_df, cat_var, 'cat')
    final_df = aggr_dummy_cols(df, final_df, colnames_dum, 'dum')
    final_df = aggr_dummy_cols(df, final_df, colnames_out)
    
    return final_df

    

In [17]:
process_violation(viol, '2010', '2016')

Index(['ENF_RESPONSE_POLICY_CODE_HPV'], dtype='object')
Index(['POLLUTANT_CODES_10193', 'POLLUTANT_CODES_10214',
       'POLLUTANT_CODES_10243', 'POLLUTANT_CODES_10305',
       'POLLUTANT_CODES_10358', 'POLLUTANT_CODES_10361',
       'POLLUTANT_CODES_10366', 'POLLUTANT_CODES_10373',
       'POLLUTANT_CODES_10461', 'POLLUTANT_CODES_10486',
       ...
       'POLLUTANT_CODES_300000452', 'POLLUTANT_CODES_5409',
       'POLLUTANT_CODES_5474', 'POLLUTANT_CODES_5617', 'POLLUTANT_CODES_603',
       'POLLUTANT_CODES_6562', 'POLLUTANT_CODES_6696', 'POLLUTANT_CODES_7003',
       'POLLUTANT_CODES_8240', 'POLLUTANT_CODES_None'],
      dtype='object', length=119)


MemoryError: 

In [19]:
inspection.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,STATE_EPA_FLAG,COMP_MONITOR_TYPE_CODE,PROGRAM_CODES
0,MI00000000000A1932,2015-06-09 00:00:00,2015,E,PCE,"CAAFESOP, CAASIP"
1,CASJV00006029S2076,2015-06-10 00:00:00,2015,L,FOO,CAATVP
2,CASJV00006039C2587,2015-06-08 00:00:00,2015,L,FOO,CAASIP
3,CASJV00006029S1234,2015-06-10 00:00:00,2015,L,FOO,CAATVP
4,CASJV00006099N7478,2015-06-11 00:00:00,2015,L,FOO,CAATVP


In [20]:
insp = inspection

In [22]:
def process_inspection(inspection_df, start_year, end_year):
    final_df = pd.DataFrame()
    
    cat_var = ['STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE']
    dum_var = ['PROGRAM_CODES']
    
    df = inspection_df[inspection_df['ACTUAL_END_DATE_year'] >= start_year]
    df = df[df['ACTUAL_END_DATE_year'] <= end_year]
    
    ## Replace NaN with 'None' (string) --> Making 'None' it's own category
    nan = df.columns[df.isnull().any()].tolist()
    values = ['None']*len(nan)
    replace_with_value(df, nan, values)
    
    ## Dummitize 
    df, colnames_dum = add_dummy(df, dum_var, sep_char = ',')
    
    #print(colnames_out)
    print(colnames_dum)
    
    final_df = aggr_dummy_cols(df, final_df, cat_var, 'cat')
    final_df = aggr_dummy_cols(df, final_df, colnames_dum, 'dum')
    
    return final_df


In [23]:
process_inspection(insp, '2010', '2016')

Index(['PROGRAM_CODES_ CAACFC', 'PROGRAM_CODES_ CAAFESOP',
       'PROGRAM_CODES_ CAAFIP', 'PROGRAM_CODES_ CAAGACTM',
       'PROGRAM_CODES_ CAAGHG', 'PROGRAM_CODES_ CAAHAPS',
       'PROGRAM_CODES_ CAAMACT', 'PROGRAM_CODES_ CAANAAQS',
       'PROGRAM_CODES_ CAANAM', 'PROGRAM_CODES_ CAANEPA',
       'PROGRAM_CODES_ CAANESH', 'PROGRAM_CODES_ CAANFRP',
       'PROGRAM_CODES_ CAANSP', 'PROGRAM_CODES_ CAANSPS',
       'PROGRAM_CODES_ CAANSPSM', 'PROGRAM_CODES_ CAANSR',
       'PROGRAM_CODES_ CAAOP', 'PROGRAM_CODES_ CAAOTH',
       'PROGRAM_CODES_ CAAPARGDC', 'PROGRAM_CODES_ CAAPSD',
       'PROGRAM_CODES_ CAAPSDPR', 'PROGRAM_CODES_ CAARMP',
       'PROGRAM_CODES_ CAASFP', 'PROGRAM_CODES_ CAASIP',
       'PROGRAM_CODES_ CAASO', 'PROGRAM_CODES_ CAASPP',
       'PROGRAM_CODES_ CAASWFC', 'PROGRAM_CODES_ CAATIP',
       'PROGRAM_CODES_ CAATVP', 'PROGRAM_CODES_ CAATVS', 'PROGRAM_CODES_CAAAR',
       'PROGRAM_CODES_CAACCP', 'PROGRAM_CODES_CAACFC', 'PROGRAM_CODES_CAAEP',
       'PROGRAM_CODES_CAAF

,PGM_SYS_ID,STATE_EPA_FLAG_E,STATE_EPA_FLAG_L,STATE_EPA_FLAG_S,COMP_MONITOR_TYPE_CODE_FFO,COMP_MONITOR_TYPE_CODE_FOO,COMP_MONITOR_TYPE_CODE_PCE,COMP_MONITOR_TYPE_CODE_PFF,COMP_MONITOR_TYPE_CODE_POC,COMP_MONITOR_TYPE_CODE_POF,...,PROGRAM_CODES_CAAPARGDC,PROGRAM_CODES_CAAPSD,PROGRAM_CODES_CAARMP,PROGRAM_CODES_CAASIP,PROGRAM_CODES_CAASO,PROGRAM_CODES_CAASPP,PROGRAM_CODES_CAASWFC,PROGRAM_CODES_CAATIP,PROGRAM_CODES_CAATVP,PROGRAM_CODES_None
0,020000000003605590,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,020000003400500100,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,020000003400546143,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,020000003400710001,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,020000003400900010,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,020000003401300001,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,020000003401300014,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,020000003401305391,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,020000003401380002,2,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,020000003401380036,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
titlev.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,COMP_MONITOR_TYPE_CODE,FACILITY_RPT_DEVIATION_FLAG
0,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
1,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
2,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
3,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y
4,UT0000004903500210,2015-04-30 00:00:00,2015,TVA,Y


In [34]:
titl = titlev

In [32]:
titlev.isnull().sum()

PGM_SYS_ID                          0
ACTUAL_END_DATE                     0
ACTUAL_END_DATE_year                0
COMP_MONITOR_TYPE_CODE              0
FACILITY_RPT_DEVIATION_FLAG    748795
dtype: int64

In [41]:
def process_titlev(titlev_df, start_year, end_year):
    final_df = pd.DataFrame()
    
    cat_var = ['COMP_MONITOR_TYPE_CODE']
    bim_var = ['FACILITY_RPT_DEVIATION_FLAG']
    
    df = titlev_df[titlev_df['ACTUAL_END_DATE_year'] >= start_year]
    df = df[df['ACTUAL_END_DATE_year'] <= end_year]
    
    ## Replace NaN with 0 (string) --> THIS ONLY APPLIES TO THE BIM_VAR!!
    nan = df.columns[df.isnull().any()].tolist()
    values = ['N']*len(nan)
    replace_with_value(df, nan, values)
    
    ## Dummitize 
    df = generate_continous_variable(df, bim_var)
    
    #print(df.groupby(bim_var[0]).groups.keys())
    
    #print(colnames_out)
    #print(colnames_dum)
    
    final_df = aggr_dummy_cols(df, final_df, cat_var, 'cat')
    final_df = aggr_dummy_cols(df, final_df, bim_var, 'bim')
    
    return final_df

In [42]:
process_titlev(titl, '2010', '2016')

,PGM_SYS_ID,COMP_MONITOR_TYPE_CODE_TVA,FACILITY_RPT_DEVIATION_FLAG
0,080000003800500001,5,5
1,080000004904500088,5,0
2,080000004904700001,5,0
3,080000004904700004,15,0
4,080000004904700023,5,0
5,080000004904700114,5,0
6,080000004904701010,10,0
7,100000000001,5,5
8,100000001600500049,10,0
9,100000001600900001,5,0


In [28]:
stacktest.head()

,PGM_SYS_ID,ACTUAL_END_DATE,ACTUAL_END_DATE_year,COMP_MONITOR_TYPE_CODE,POLLUTANT_CODES,AIR_STACK_TEST_STATUS_CODE
0,RI0000004400900016,2007-10-03 00:00:00,2007,CST,Volatile organic compounds,NaN
1,CT0000000900900001,2007-01-17 00:00:00,2007,CST,Volatile organic compounds,NaN
2,CT0000000900900001,2007-01-18 00:00:00,2007,CST,Volatile organic compounds,NaN
3,ME0000002301300004,2007-05-30 00:00:00,2007,CST,Volatile organic compounds,NaN
4,MA0000002511900605,2011-09-28 00:00:00,2011,CST,VOLATILE ORGANIC COMPOUNDS (VOCS),PSS


In [49]:
stack = stacktest

In [47]:
stacktest['POLLUTANT_CODES'].value_counts()

Carbon monoxide                                                           30977
NITROGEN OXIDES NO2                                                       29870
TOTAL PARTICULATE MATTER                                                  23628
VOLATILE ORGANIC COMPOUNDS (VOCS)                                         20984
FACIL                                                                     10734
NITROGEN OXIDES                                                           10258
PARTICULATE MATTER < 10 UM                                                 9398
Sulfur dioxide                                                             8211
VISIBLE EMISSIONS                                                          6696
NITROGEN OXIDES NO                                                         2164
Hydrochloric acid                                                          2095
Formaldehyde                                                               1942
Lead                                    

In [48]:
stacktest.isnull().sum()

PGM_SYS_ID                        0
ACTUAL_END_DATE                   0
ACTUAL_END_DATE_year              0
COMP_MONITOR_TYPE_CODE            0
POLLUTANT_CODES               64862
AIR_STACK_TEST_STATUS_CODE     8900
dtype: int64

In [50]:
def process_stacktest(stacktest_df, start_year, end_year):
    final_df = pd.DataFrame()
    
    #I HAVEN'T PUT IN POLLUTANT_CODES
    cat_var = ['AIR_STACK_TEST_STATUS_CODE', 'COMP_MONITOR_TYPE_CODE']
    
    df = stacktest_df[stacktest_df['ACTUAL_END_DATE_year'] >= start_year]
    df = df[df['ACTUAL_END_DATE_year'] <= end_year]
    
    ## Replace NaN with 0 (string) --> THIS ONLY APPLIES TO THE BIM_VAR!!
    nan = df.columns[df.isnull().any()].tolist()
    values = ['None']*len(nan)
    replace_with_value(df, nan, values)
    
    ## Dummitize --> Not really needed here 
    
    final_df = aggr_dummy_cols(df, final_df, cat_var, 'cat')
    
    return final_df

In [51]:
process_stacktest(stack, '2010', '2016')

,PGM_SYS_ID,AIR_STACK_TEST_STATUS_CODE_FAI,AIR_STACK_TEST_STATUS_CODE_INC,AIR_STACK_TEST_STATUS_CODE_None,AIR_STACK_TEST_STATUS_CODE_PEN,AIR_STACK_TEST_STATUS_CODE_PSS,COMP_MONITOR_TYPE_CODE_CST
0,020000003400190019,0,0,1,0,0,1
1,020000003400309989,0,0,1,0,0,1
2,020000003400590003,0,0,1,0,0,1
3,020000003400780007,0,0,1,0,0,1
4,020000003401390003,0,0,1,0,0,1
5,020000003402390003,0,0,1,0,0,1
6,020000003402570003,0,0,1,0,0,1
7,020000003402790002,0,0,1,0,0,1
8,020000003403180000,0,0,2,0,0,2
9,020000003403190001,0,0,1,0,0,1


In [58]:
formalact.head()

,PGM_SYS_ID,SETTLEMENT_ENTERED_DATE,SETTLEMENT_ENTERED_DATE_year,ENF_TYPE_CODE,PENALTY_AMOUNT
2,IL000197800AAA,2009-01-27,2009,CIV,0.0
4,IL000021816AAQ,2015-04-07,2015,CIV,0.0
5,OH0000000374010102,2015-04-07,2015,CIV,0.0
6,IL000143065AFM,2015-04-07,2015,CIV,0.0
7,IL000095020ABS,2015-04-07,2015,CIV,0.0


In [72]:
formalact.isnull().sum()

PGM_SYS_ID                      0
SETTLEMENT_ENTERED_DATE         0
SETTLEMENT_ENTERED_DATE_year    0
ENF_TYPE_CODE                   0
PENALTY_AMOUNT                  0
dtype: int64

In [73]:
def process_formalact(formalact_df, start_year, end_year):
    final_df = pd.DataFrame()
    
    cat_var = ['ENF_TYPE_CODE']
    cont_var = ['PENALTY_AMOUNT']
    
    df = formalact_df[formalact_df['ACTUAL_END_DATE_year'] >= start_year]
    df = df[df['ACTUAL_END_DATE_year'] <= end_year]
    
    ## Replace NaN with 0 (string) --> No need for this but ill leave it here
    nan = df.columns[df.isnull().any()].tolist()
    values = ['None']*len(nan)
    replace_with_value(df, nan, values)
    
    ## Dummitize --> Not really needed here 
    
    final_df = aggr_dummy_cols(df, final_df, cat_var, 'cat')
    #NEED TO ADD THE SUM of the amount!
    
    return final_df

In [30]:
informalact.head()

,PGM_SYS_ID,ACHIEVED_DATE,ACHIEVED_DATE_year,ENF_TYPE_CODE
84,AR0000000506900118,2007-08-10 00:00:00,2007,NOV
86,TX0000004844900005,2008-02-05 00:00:00,2008,NOV
87,LA0000002200500046,2008-06-26 00:00:00,2008,NOV
89,0900000006071R9800,2008-03-18 00:00:00,2008,NOV
90,0900000006099R9800,2008-03-08 00:00:00,2008,NOV


In [74]:
def process_informalact(informalact_df, start_year, end_year):
    final_df = pd.DataFrame()
    
    cat_var = ['ENF_TYPE_CODE']
    
    df = informalact_df[informalact_df['ACTUAL_END_DATE_year'] >= start_year]
    df = df[df['ACTUAL_END_DATE_year'] <= end_year]
    
    ## Replace NaN with 0 (string) --> No need for this but ill leave it here
    nan = df.columns[df.isnull().any()].tolist()
    values = ['None']*len(nan)
    replace_with_value(df, nan, values)
    
    ## Dummitize --> Not really needed here 
    
    final_df = aggr_dummy_cols(df, final_df, cat_var, 'cat')
    
    return final_df

In [76]:
#### AGGREGATE EVERYTHING WITH FACILITIES HEREEE ####